In [1]:
#Problema MBA
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

m = gp.Model('problema_Adilson')

Set parameter Username


In [2]:
Cidades_origem = ['Araraquara', 'SJC']

In [3]:
CD = ['CD_Araraquara', 'CD_SJC']

In [4]:
Mercado = ['SP', 'BH', 'RJ']

In [5]:
path = '/Users/TullioPires/Python-Mestrado/cidade-CD.csv'
custos = pd.read_csv(path, index_col=[0,1], squeeze=True)

#Pivotar a tabela, para ler
custos.reset_index().pivot(index = 'Origem', columns = 'Destino', values = 'Custo')

/var/folders/f8/sx2sm28s5xn92g54_n5vypwh0000gn/T/ipykernel_72451/3608632233.py:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  custos = pd.read_csv(path, index_col=[0,1], squeeze=True)


Destino,CD1,CD2
Origem,,
Araraquara,23.5,51.4
SJC,9.4,54.0


In [6]:
caminho = '/Users/TullioPires/Python-Mestrado/CD-destino.csv'
custos2 = pd.read_csv(caminho, index_col=[0,1], squeeze=True)

#Pivotar a tabela, para ler
custos2.reset_index().pivot(index = 'Origem', columns = 'Destino', values = 'Custo')

/var/folders/f8/sx2sm28s5xn92g54_n5vypwh0000gn/T/ipykernel_72451/438711658.py:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  custos2 = pd.read_csv(caminho, index_col=[0,1], squeeze=True)


Destino,BH,RJ,SP
Origem,,,
CD1,59.8,70.8,27.7
CD2,52.0,34.1,10.4


In [7]:
oferta = pd.Series([800, 100], index=CD, name='oferta')
oferta.to_frame()

,oferta
CD_Araraquara,800
CD_SJC,100


In [8]:
demanda = pd.Series([500, 400, 900], index=Mercado, name='demanda')
demanda.to_frame()

,demanda
SP,500
BH,400
RJ,900


In [9]:
m = gp.Model('problema_Adilson')
x = m.addVars(Cidades_origem, CD, vtype = GRB.CONTINUOUS, name = 'cidade_CD')
y = m.addVars(CD, Mercado, vtype=GRB.CONTINUOUS, name='CD_destino')
m.update()
m.write('problema_Adilson.lp')

In [10]:
#Criando as restrições de oferta
restricao_oferta_araquara = m.addConstr((x['Araraquara', 'CD_Araraquara'] +
                                         x['Araraquara', 'CD_SJC']== 800),
                                        name='restrição de oferta Araraquara')

restricao_oferta_SJC = m.addConstr((x['SJC', 'CD_Araraquara'] + x['SJC', 'CD_SJC'])==1000,
                                  name = 'restrição de oferta SJC')
m.update()

In [11]:
#Criando as restrições demanda
restricao_demanda_SP = m.addConstr((y['CD_Araraquara','SP']
                                                  +y['CD_SJC', 'SP'] == 500),
                                                name='restrição de demanda SP')
restricao_demanda_BH = m.addConstr((y['CD_Araraquara','BH']
                                                +y['CD_SJC', 'BH'] == 400),
                                               name='restrição demanda BH')
restricao_demanda_RJ = m.addConstr((y['CD_Araraquara', 'RJ']
                                  +y['CD_SJC', 'RJ'] == 900),
                                 name='restrição demanda RJ')

m.update()

In [12]:
#Criando a função objetivo
m.setObjective(23.5*x['Araraquara', 'CD_Araraquara']+51.4*x['Araraquara', 'CD_SJC']
               +9.4*x['SJC', 'CD_Araraquara']+54*x['SJC', 'CD_SJC']+27.7*y['CD_Araraquara','SP']
               +59.8*y['CD_SJC', 'SP']+70.8*y['CD_Araraquara','BH']
               +10.4*y['CD_SJC', 'BH']+52*y['CD_Araraquara', 'RJ']
               +34.1*y['CD_SJC', 'RJ'], GRB.MINIMIZE)

In [13]:
m.write('problema_Adilson.lp')

In [14]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x66737518
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve removed 5 rows and 10 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6900000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.690000000e+04


In [29]:
#Obtendo os valores de X
#xvals = {k: v.x for k,v in x.items() if v.x> 0}
#xvals
x.items()

[(('Araraquara', 'CD_Araraquara'),
  <gurobi.Var cidade_CD[Araraquara,CD_Araraquara] (value 800.0)>),
 (('Araraquara', 'CD_SJC'),
  <gurobi.Var cidade_CD[Araraquara,CD_SJC] (value 0.0)>),
 (('SJC', 'CD_Araraquara'),
  <gurobi.Var cidade_CD[SJC,CD_Araraquara] (value 1000.0)>),
 (('SJC', 'CD_SJC'), <gurobi.Var cidade_CD[SJC,CD_SJC] (value 0.0)>)]

In [28]:
#Obtendo os valors de Y
y.items()

[(('CD_Araraquara', 'SP'),
  <gurobi.Var CD_destino[CD_Araraquara,SP] (value 500.0)>),
 (('CD_Araraquara', 'BH'),
  <gurobi.Var CD_destino[CD_Araraquara,BH] (value 0.0)>),
 (('CD_Araraquara', 'RJ'),
  <gurobi.Var CD_destino[CD_Araraquara,RJ] (value 0.0)>),
 (('CD_SJC', 'SP'), <gurobi.Var CD_destino[CD_SJC,SP] (value 0.0)>),
 (('CD_SJC', 'BH'), <gurobi.Var CD_destino[CD_SJC,BH] (value 400.0)>),
 (('CD_SJC', 'RJ'), <gurobi.Var CD_destino[CD_SJC,RJ] (value 900.0)>)]

In [18]:
#Mostrando o resultado da função objetivo
obj = m.getObjective()
print(f"O valor da função objetivo é {round(obj.getValue(),2)}")

O valor da função objetivo é 76900.0
